#### TODO:

- make cash out function, that is, sell all holdings on last entry in dataframe.

In [1]:
import pandas as pd
import numpy as np
import math

## Mock DATA

In [2]:
dates = pd.date_range('1/1/2000', periods=50)
mock_frame = pd.DataFrame(index=dates)
mock_frame["close"] = np.random.rand(len(mock_frame), 1)
mock_frame["random_forest"] = np.random.choice(["buy", "sell", "hold"], len(mock_frame))
mock_frame["svc"] = np.random.choice(["buy", "sell", "hold"], len(mock_frame))
mock_frame["logreg"] = np.random.choice(["buy", "sell", "hold"], len(mock_frame))
mock_frame["gauss_nb"] = np.random.choice(["buy", "sell", "hold"], len(mock_frame))

### Help Functions

In [3]:
def calculate_amount(unit_price, budget):
    amount = budget / unit_price
    return amount

def calculate_profit(buy_price, sell_price):
    profit = sell_price - buy_price
    return profit

## Single position investment function

In [4]:
def single_position_investing(input_dataframe, investment_capital):
    
    res = pd.DataFrame(index=mock_frame.index, columns=mock_frame.columns[1:])
    res = res.rename(columns={"random_forest": "random_forest_profit", "svc":"svc_profit",
                              "logreg":"logreg_profit", "gauss_nb": "gauss_nb_profit"})
    
    # LOOP THROUGH COLUMNS
    for col in input_dataframe.columns[1:]:
        current_profit = 0
        holding = False
        holding_close = 0
        holding_value = 0
        holding_amount = 0
        resulting_col = col + "_profit"
        
        wallet = investment_capital
    
        # LOOP THROUGH EACH ROW
        for i in range(len(input_dataframe)):
            close_price = input_dataframe["close"].iloc[i]
            label = input_dataframe[col].iloc[i]
            
            # ON BUY
            if((holding == False) & (label == "buy")):
                
                # calc amount that can be bought at daily close price
                holding_amount = calculate_amount(close_price, wallet)
                # remove money from wallet
                wallet -= wallet
                # update holding_close to keep track of buy price
                holding_close = close_price
                
                # ADD NEW PROFIT TO DATAFRAME
                res[resulting_col].iloc[i] = current_profit

                # UPDATE HOLDING VALUES
                holding = True

            # ON SELL
            elif((holding == True) & (label == "sell")):

                # get sell price and compare to buy price
                selling_price = close_price * holding_amount
                buying_price = holding_close * holding_amount
                # update wallet to have money again, i.e selling_price
                wallet = selling_price
                # register profit/loss
                profit = calculate_profit(buying_price, selling_price)
                current_profit += profit
                res[resulting_col].iloc[i] = current_profit
                holding_amount = 0
                holding = False

            # ON HOLD
            else:

                # ADD NEW PROFIT TO DATAFRAME
                res[resulting_col].iloc[i] = current_profit
    
    output_dataframe = pd.concat([input_dataframe, res], axis=1)
    
    return output_dataframe

## Multi position investment function

In [32]:
def multi_position_investing(input_dataframe, investment_capital, bucket_size):
    
    res = pd.DataFrame(index=mock_frame.index, columns=mock_frame.columns[1:])
    res = res.rename(columns={"random_forest": "random_forest_profit", "svc":"svc_profit",
                              "logreg":"logreg_profit", "gauss_nb": "gauss_nb_profit"})
    
    # LOOP THROUGH COLUMNS
    for col in input_dataframe.columns[1:]:
        current_profit = 0
        holding_value = 0
        resulting_col = col + "_profit"
        
        wallet = investment_capital
        positions_close = []
        positions_amount = []
    
        # LOOP THROUGH EACH ROW
        for i in range(len(input_dataframe)):
            investment_amount = wallet / bucket_size
            current_close_price = input_dataframe["close"].iloc[i]
            label = input_dataframe[col].iloc[i]
            
            # ON BUY
            if((wallet > 0) & (label == "buy")):
                # check to see if invest wallet amount or investment amount.
                if wallet < investment_amount:
                    amount = calculate_amount(current_close_price, wallet)
                    positions_close.append(current_close_price)
                    positions_amount.append(amount)
                    # remove money from wallet
                    wallet -= wallet
                else:
                    amount = calculate_amount(current_close_price, investment_amount)
                    positions_close.append(current_close_price)
                    positions_amount.append(amount)
                    # remove money from wallet
                    wallet -= investment_amount
                # ADD NEW PROFIT TO DATAFRAME
                res[resulting_col].iloc[i] = current_profit

            # ON SELL
            elif((len(positions_close) > 0) & (label == "sell")):
                for close_p, amount_p in zip(positions_close, positions_amount):
                    # get sell price and compare to buy price
                    selling_price = current_close_price * amount_p
                    buying_price = close_p * amount_p
                    # update wallet to have money again, i.e selling_price
                    wallet += selling_price
                    # register profit/loss
                    profit = calculate_profit(buying_price, selling_price)
                    current_profit += profit
                    
                # ADD NEW PROFIT TO DATAFRAME
                res[resulting_col].iloc[i] = current_profit
                
                # EMPTY POSITIONS LISTS
                positions_close = []
                positions_amount = []

            # ON HOLD
            else:
                # Since we are holding, do nothing except udpate current profit
                # ADD NEW PROFIT TO DATAFRAME
                res[resulting_col].iloc[i] = current_profit
                
    output_dataframe = pd.concat([input_dataframe, res], axis=1)
    
    return output_dataframe

### Example use Single:

In [40]:
results = single_position_investing(mock_frame, 100)
results.tail(1)

,close,random_forest,svc,logreg,gauss_nb,random_forest_profit,svc_profit,logreg_profit,gauss_nb_profit
2000-02-19,0.699117,sell,sell,sell,buy,303.212,8120.22,-98.8855,94.1511


### Example use Multi:

In [41]:
results2 = multi_position_investing(mock_frame, 100,1)
results2.tail(1)

,close,random_forest,svc,logreg,gauss_nb,random_forest_profit,svc_profit,logreg_profit,gauss_nb_profit
2000-02-19,0.699117,sell,sell,sell,buy,316.788,1967.98,-98.8855,71.7711


In [43]:
view_res = pd.DataFrame(results['random_forest_profit'])
view_res['random_forest_profit2'] = results2['random_forest_profit']
view_res.tail()

,random_forest_profit,random_forest_profit2
2000-02-15,455.53,328.956
2000-02-16,455.53,328.956
2000-02-17,455.53,328.956
2000-02-18,455.53,328.956
2000-02-19,303.212,316.788
